# NLP topic modeling using LDA, HdpModel

In this notebook, I have done topic modeling on description feature in the data using LDA. After finding optimum number of topics using HDP model I used that number of topics in LDA and created new derived feature in the data called topic(consisting of topic number for each entry in description Feature) for the final model.

In [1]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geolocator = Nominatim(user_agent="Optima")
import json
import pandas as pd
import requests
import csv


#from fastai.imports import *
#from fastai.structured import *
#from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import os
import re


#!C:/Users/sdk19/anaconda3/python -m pip install wordcloud <-- Installation required
#!C:/Users/sdk19/anaconda3/python -m pip install gensim <-- Installation required
#!C:/Users/sdk19/anaconda3/python -m pip install spacy <-- Installation required    
    
from wordcloud import WordCloud
import gensim
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
    
#np.set_printoptions(threshold=50, edgeitems=20)

#PATH='amsterdam/'

from IPython.display import HTML, display
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from gensim.models import HdpModel
import logging
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdk19\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv("C:\\Users\\sdk19\\OneDrive\\Desktop\\umd sem2\\data mining\\project\\airbnbTrain.csv")
pd.set_option('display.max_columns', 66) # to display all the columns of dataframe
df.head()

,id,high_booking_rate,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,extra_people,guests_included,host_about,host_acceptance_rate,host_has_profile_pic,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_neighbourhood,host_response_rate,host_response_time,host_since,host_verifications,house_rules,instant_bookable,interaction,is_business_travel_ready,is_location_exact,latitude,longitude,market,maximum_nights,minimum_nights,monthly_price,neighborhood_overview,neighbourhood,notes,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,security_deposit,space,square_feet,state,transit,weekly_price,zipcode,{randomControl}
0,1060760,0,NaN,4,"{TV,""Cable TV"",Wifi,""Air conditioning"",Pool,Ki...",19,67,29,29,2.0,Real Bed,2.0,3.0,strict_14_with_grace_period,Hallandale Beach,$140.00,Enjoy an amazing water view from this beautifu...,$0.00,1,NaN,NaN,True,False,False,9.0,"Miami, Florida, United States",NaN,0.95,within an hour,43538.0,"['email', 'phone', 'offline_government_id', 'g...",NaN,True,I am totally available via text or phone to ma...,False,True,25.98676,-80.12252,Miami,30,1,NaN,NaN,NaN,The Beachwalk Resort asks $100 for deposit.,$199.00,Serviced apartment,False,False,False,10.0,9.0,9.0,10.0,10.0,90.0,10.0,Entire home/apt,NaN,The apartment is located at the Beachwalk Reso...,NaN,FL,NaN,NaN,33009,113890
1,1040685,0,NaN,4,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",0,0,0,0,1.0,Real Bed,2.0,2.0,strict_14_with_grace_period,New York,$30.00,A New York charmer on the edge of Times Square...,$0.00,4,"Hello! It's me! I grew up in Utah, I had a hum...",NaN,True,False,False,1.0,"New York, New York, United States",Hell's Kitchen,NaN,NaN,41807.0,"['email', 'phone', 'reviews']",Please remember that this is our home. We are ...,False,NaN,False,True,40.76198,-73.98718,New York,14,2,NaN,NaN,Midtown,This is on the 5th floor of a walk up. The fir...,$350.00,Apartment,False,False,False,10.0,10.0,9.0,10.0,10.0,98.0,9.0,Entire home/apt,NaN,You will not find a more adorable apartment! A...,NaN,NY,This apartment is in one of the best locations...,NaN,10019,116337
2,1001074,0,NaN,4,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",0,0,0,0,1.0,Real Bed,2.0,2.0,moderate,Los Angeles,$125.00,Charming & private 2 bedroom attached guest ho...,$0.00,3,"My partner, Art Toye and I live in Los Angeles...",NaN,True,True,True,3.0,"Los Angeles, California, United States",Mar Vista,1.00,within an hour,41637.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",NaN,True,"We are usually at home, available to answer qu...",False,True,34.00697,-118.45050,Los Angeles,29,2,NaN,You probably will be surprised that such a cal...,Mar Vista,NaN,$125.00,House,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire home/apt,$200.00,NaN,NaN,CA,NaN,NaN,90066,112297
3,1197330,0,NaN,2,"{Wifi,""Air conditioning"",""Free parking on prem...",24,173,54,84,2.5,Real Bed,1.0,1.0,moderate,Lino Lakes,NaN,Quiet Neighborhood I work mondays - Fridays bu...,$0.00,1,NaN,NaN,True,False,False,0.0,"Akron, Ohio, United States",NaN,NaN,NaN,43119.0,"['email', 'phone']",NaN,True,I work mondays - Fridays but I’m home most of ...,False,True,45.18900,-93.13541,Minneapolis,1125,2,NaN,NaN,NaN,NaN,$35.00,Townhouse,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private room,NaN,NaN,NaN,MN,"House is 24 mins from downtown, it’s a quiet s...",NaN,55014,125404
4,1031737,0,Access the entire house. Garage will be locked.,2,"{TV,Wifi,Kitchen,Heating,Washer,Dryer,""Smoke d...",4,64,34,64,1.0,Real Bed,1.0,1.0,strict_14_with_grace_period,Long Beach,NaN,Once you get unpacked and settled in you'll pr...,$0.00,1,My personal mission: To enrich live

In [3]:
stop_words = stopwords.words('english')# to get rid of english stowords from describtion

In [4]:
# replacing new line with blank
def strip_newline(series):
    return [review.replace('\n','') for review in series]

In [5]:
df['description'] = strip_newline(df.description.astype('str'))

In [6]:
#This function will lowercase,tokenize and de-accents words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [7]:
words_tr = list(sent_to_words(df.description))

In [9]:
#Next will remove stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
words_tr = remove_stopwords(words_tr)

In [10]:
#Automatically detect common phrases – aka multi-word expressions, word n-gram collocations – from a stream of sentences.
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

In [11]:
bigram_tr, trigram_tr = bigrams(words_tr)

In [12]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [13]:
trigrams_tr = [trigram_tr[bigram_tr[review]] for review in words_tr]

In [14]:
#grouping together the different inflected forms of a word so they can be analysed as a single item.
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
lemma_lg = lemmatization(trigrams_tr)

In [19]:
id2word_lg = gensim.corpora.Dictionary(lemma_lg)#Dictionary encapsulates the mapping between normalized words and their integer ids.
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
#id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]#converts a collection of words to its bag-of-words representation: a list of (word_id, word_frequency) 2-tuples.

In [20]:
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)#to infers the number of topics from the description column using HdpModel

In [21]:
len(hdp.print_topics())

20

According to HdpModel optimal number of topics to use are 20. Thus we can now run lda with these number of topics to assign topic for each listing.
Now Following the same processing done before for description column.

In [22]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [23]:

#stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
#                  'would','really','like','great','service','came','got'])


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
    
def get_corpus(df):
    df['description'] = strip_newline(df.description.astype(str))
    words = list(sent_to_words(df.description))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

train_corpus, train_id2word, bigram_train = get_corpus(df)

In [24]:
#running lda after following same text preprocessing on description column with 20 number of topics.
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=20,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [25]:
lda_train4.print_topics(10,num_words=15)[:10]

[(15,
  '0.030*"downtown" + 0.025*"restaurants" + 0.021*"located" + 0.020*"city" + 0.017*"heart" + 0.016*"one" + 0.015*"neighborhood" + 0.014*"blocks" + 0.014*"walk" + 0.013*"street" + 0.013*"historic" + 0.012*"location" + 0.011*"distance" + 0.011*"nashville" + 0.010*"within_walking"'),
 (2,
  '0.013*"loft" + 0.012*"space" + 0.008*"light" + 0.008*"windows" + 0.008*"art" + 0.008*"built" + 0.008*"historic" + 0.007*"old" + 0.006*"style" + 0.006*"unique" + 0.006*"original" + 0.006*"vintage" + 0.005*"modern" + 0.004*"walls" + 0.004*"classic"'),
 (16,
  '0.056*"beach" + 0.021*"condo" + 0.016*"ocean" + 0.013*"unit" + 0.012*"located" + 0.011*"resort" + 0.011*"one" + 0.011*"enjoy" + 0.011*"beautiful" + 0.011*"lanai" + 0.010*"pool" + 0.009*"property" + 0.008*"views" + 0.008*"beaches" + 0.008*"walk"'),
 (5,
  '0.023*"close" + 0.021*"restaurants" + 0.018*"walk" + 0.018*"neighborhood" + 0.016*"place" + 0.016*"min" + 0.015*"great" + 0.014*"available" + 0.013*"downtown" + 0.011*"walking_distance" + 0

In [27]:
train_vecs = []
for i in range(len(df)):
    top_topics = lda_train4.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    #topic_vec.extend([df.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(df.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)
    
    

In [29]:
df['topic']=train_vecs
#df.topic.head()
topic=[]
for i,r in df.iterrows():
    
    max_index=train_vecs[i].index(max(train_vecs[i]))
    topic.append(max_index)

df['topic']=topic
df.head()

,id,high_booking_rate,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,extra_people,guests_included,host_about,host_acceptance_rate,host_has_profile_pic,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_neighbourhood,host_response_rate,host_response_time,host_since,host_verifications,house_rules,instant_bookable,...,is_business_travel_ready,is_location_exact,latitude,longitude,market,maximum_nights,minimum_nights,monthly_price,neighborhood_overview,neighbourhood,notes,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,security_deposit,space,square_feet,state,transit,weekly_price,zipcode,{randomControl},topic
0,1060760,0,NaN,4,"{TV,""Cable TV"",Wifi,""Air conditioning"",Pool,Ki...",19,67,29,29,2.0,Real Bed,2.0,3.0,strict_14_with_grace_period,Hallandale Beach,$140.00,Enjoy an amazing water view from this beautifu...,$0.00,1,NaN,NaN,True,False,False,9.0,"Miami, Florida, United States",NaN,0.95,within an hour,43538.0,"['email', 'phone', 'offline_government_id', 'g...",NaN,True,...,False,True,25.98676,-80.12252,Miami,30,1,NaN,NaN,NaN,The Beachwalk Resort asks $100 for deposit.,$199.00,Serviced apartment,False,False,False,10.0,9.0,9.0,10.0,10.0,90.0,10.0,Entire home/apt,NaN,The apartment is located at the Beachwalk Reso...,NaN,FL,NaN,NaN,33009,113890,16
1,1040685,0,NaN,4,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",0,0,0,0,1.0,Real Bed,2.0,2.0,strict_14_with_grace_period,New York,$30.00,A New York charmer on the edge of Times Square...,$0.00,4,"Hello! It's me! I grew up in Utah, I had a hum...",NaN,True,False,False,1.0,"New York, New York, United States",Hell's Kitchen,NaN,NaN,41807.0,"['email', 'phone', 'reviews']",Please remember that this is our home. We are ...,False,...,False,True,40.76198,-73.98718,New York,14,2,NaN,NaN,Midtown,This is on the 5th floor of a walk up. The fir...,$350.00,Apartment,False,False,False,10.0,10.0,9.0,10.0,10.0,98.0,9.0,Entire home/apt,NaN,You will not find a more adorable apartment! A...,NaN,NY,This apartment is in one of the best locations...,NaN,10019,116337,13
2,1001074,0,NaN,4,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",0,0,0,0,1.0,Real Bed,2.0,2.0,moderate,Los Angeles,$125.00,Charming & private 2 bedroom attached guest ho...,$0.00,3,"My partner, Art Toye and I live in Los Angeles...",NaN,True,True,True,3.0,"Los Angeles, California, United States",Mar Vista,1.00,within an hour,41637.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",NaN,True,...,False,True,34.00697,-118.45050,Los Angeles,29,2,NaN,You probably will be surprised that such a cal...,Mar Vista,NaN,$125.00,House,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire home/apt,$200.00,NaN,NaN,CA,NaN,NaN,90066,112297,1
3,1197330,0,NaN,2,"{Wifi,""Air conditioning"",""Free parking on prem...",24,173,54,84,2.5,Real Bed,1.0,1.0,moderate,Lino Lakes,NaN,Quiet Neighborhood I work mondays - Fridays bu...,$0.00,1,NaN,NaN,True,False,False,0.0,"Akron, Ohio, United States",NaN,NaN,NaN,43119.0,"['email', 'phone']",NaN,True,...,False,True,45.18900,-93.13541,Minneapolis,1125,2,NaN,NaN,NaN,NaN,$35.00,Townhouse,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private room,NaN,NaN,NaN,MN,"House is 24 mins from downtown, it’s a quiet s...",NaN,55014,125404,1
4,1031737,0,Access the entire house. Garage will be locked.,2,"{TV,Wifi,Kitchen,Heating,Washer,Dryer,""Smoke d...",4,64,34,64,1.0,Real Bed,1.0,1.0,strict_14_with_grace_period,Long Beach,NaN,Once you get unpacked and settled in you'll pr...,$0.00,1,My personal mission: To enrich lives through m...,NaN,True,False,False,1.0,"Long Beach, California, United States",Long Beach,NaN,NaN,43457.0,"['email', 'phone', 'offl

Topic column added for topics ranging from 1 to 20 for description column. I will use this column as one of the feature in my model.

In [ ]:
#df2=df[['id','topic']]
#df2.to_csv('topicsLDA.csv')